# Preparation

In [ ]:
%cd /home/dvc-2-iris-demo

In [ ]:
import yaml

# Look on pipelines config 
config = yaml.safe_load(open('config/pipeline_config.yml'))

config

# Prepare configs

In [ ]:
!python src/pipelines/prepare_configs.py \
    --config=config/pipeline_config.yml

## Browse folder with configs

In [ ]:
!ls experiments/

# Extract features

In [ ]:
# featurization config
!cat experiments/featurize_config.yml

In [ ]:
!python src/pipelines/featurize.py \
    --config=experiments/featurize_config.yml

In [ ]:
# iris dataset with new features is created
!ls data/interim

# Split train/test dataset

In [ ]:
# split config
!cat experiments/split_train_test_config.yml

In [ ]:
!python src/pipelines/split_train_test.py \
    --config=experiments/split_train_test_config.yml \
    --base_config=config/pipeline_config.yml
    

In [ ]:
# train and test datsets are created
!ls data/processed/

# Train model

In [ ]:
# train config
!cat experiments/train_config.yml

In [ ]:
!python src/pipelines/train.py \
    --config=experiments/train_config.yml \
    --base_config=config/pipeline_config.yml

In [ ]:
# model is created
!ls models/

# Evaluate model 

In [ ]:
# evaluate config
!cat experiments/evaluate_config.yml

In [ ]:
!python src/pipelines/evaluate.py \
    --config=experiments/evaluate_config.yml \
    --base_config=config/pipeline_config.yml

In [ ]:
# metrics file eval.txt is created
!ls experiments

In [ ]:
!cat experiments/eval.txt

In [ ]:
evaluate_report = yaml.load(open('experiments/eval.txt'), Loader=yaml.FullLoader)
evaluate_report